In [17]:
import sys
# setting path
sys.path.append('..')

In [18]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

from src import io, var, utils, fourier, lin_reg, reconstruction
from vis import plotter

import importlib
importlib.reload(io)
importlib.reload(var)
importlib.reload(utils)
importlib.reload(fourier)
importlib.reload(lin_reg)
importlib.reload(reconstruction)

importlib.reload(plotter)

<module 'vis.plotter' from '/home/ray/git-projects/spec_appx/notebooks/../vis/plotter.py'>

Prepare orography by generating underlying lat-lon grid of interest.

In [19]:
lat_min = 52.0
lat_max = 64.0

lon_min = -158.0
lon_max = -127.0

dlat = 0.5
dlon = 0.5

lat = np.arange(lat_min, lat_max + dlat, dlat)
lon = np.arange(lon_min, lon_max + dlon, dlon)

lat_mgrid, lon_mgrid = np.meshgrid(lat,lon)

In [20]:
grid = var.grid()
    
reader = io.ncdata()
fn = '../data/icon_grid_0012_R02B04_G_linked.nc'
reader.read_dat(fn, grid)
grid.apply_f(utils.rad2deg)

vids = []
for lat_ref in lat:
    for lon_ref in lon:
        vid = utils.pick_cell(lat_ref, lon_ref, grid)
        vids.append(vid)


In [21]:
icon_cell_indexes = np.array(list((set(vids))))

clat = grid.clat
clat_verts = grid.clat_vertices
clon = grid.clon
clon_verts = grid.clon_vertices
links = grid.links

print(clat)
print(links)

[ 88.89564383  87.21227967  88.07335469 ... -28.02779951 -28.11796307
 -27.45662172]
[[75 79 83 ...  0  0  0]
 [75 79  0 ...  0  0  0]
 [75 79 83 ...  0  0  0]
 ...
 [26 50  0 ...  0  0  0]
 [26 50  0 ...  0  0  0]
 [26 50  0 ...  0  0  0]]


In [22]:
comp_clat = clat[icon_cell_indexes]
comp_clon = clon[icon_cell_indexes]
comp_clat_verts = clat_verts[icon_cell_indexes]
comp_clon_verts = clon_verts[icon_cell_indexes]

ncfile = nc.Dataset('../data/icon_compact.nc',mode='w') 
print(ncfile)

cell = ncfile.createDimension('cell', np.size(comp_clat))     # latitude axis
nv = ncfile.createDimension('nv', 3)    # longitude axis
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact ICON grid for testing and debugging purposes'
print(ncfile.title)

clat = ncfile.createVariable('clat', np.float32, ('cell',))
clat.units = 'radian'
clat.long_name = 'center latitude'

clon = ncfile.createVariable('clon', np.float32, ('cell',))
clon.units = 'radian'
clon.long_name = 'center longitude'

clat_verts = ncfile.createVariable('clat_vertices', np.float32, ('cell','nv',))
clat_verts.units = 'radian'

clon_verts = ncfile.createVariable('clon_vertices', np.float32, ('cell','nv',))
clon_verts.units = 'radian'

clat[:] = comp_clat
clon[:] = comp_clon
clat_verts[:,:] = comp_clat_verts
clon_verts[:,:] = comp_clon_verts

print(clon_verts[:,:])

ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('cell', <class 'netCDF4._netCDF4.Dimension'>: name = 'cell', size = 120)
('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 3)
Compact ICON grid for testing and debugging purposes
[[-141.37692  -144.       -139.10022 ]
 [-126.74175  -124.161026 -127.65986 ]
 [-127.65986  -124.98467  -128.6811  ]
 [-128.6811   -131.22144  -127.65986 ]
 [-131.22144  -133.54704  -130.12047 ]
 [-130.12047  -127.65986  -131.22144 ]
 [-127.65986  -130.12047  -126.74175 ]
 [-133.54704  -131.22144  -134.83536 ]
 [-131.22144  -128.6811   -132.44728 ]
 [-132.44728  -134.83536  -131.22144 ]
 [-134.83536  -132.44728  -136.26743 ]
 [-139.10022  -136.16074  -141.37692 ]
 [-128.12617  -131.1148   -126.94688 ]
 [-126.94688  -123.78986  -128.12617 ]
 [-131.1148   -133.819    -129.8259  ]
 [-133.819    -136.26743  -132.44728 ]
 [-132.44728  -129.8259   -133

In [24]:
links_tmp = links[icon_cell_indexes].flatten()
links_tmp = links_tmp[np.where(links_tmp > 0)]
links_tmp = np.sort(list(set(links_tmp)))
links_tmp -= 1

print(links_tmp)

topo = var.topo()
fn = '../data/GMTED2010_topoGlobal_SGS_30ArcSec.nc'
reader.read_dat(fn, topo)

lon = topo.lon
lat = topo.lat
z = topo.topo

del topo

# lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

sz_tmp = np.size(links_tmp)
# print(nlat,nlon, np.size(links_tmp))

compactified_topo = np.zeros((sz_tmp,nlat, nlon))
print(compactified_topo.shape)

compactified_lat = np.zeros((sz_tmp,nlat))
compactified_lon = np.zeros((sz_tmp,nlon))
    
for i,lnk in enumerate(links_tmp):
    print("i, lnk = ", (i, lnk))
    compactified_lat[i] = lat[lnk]
    compactified_lon[i] = lon[lnk]
    compactified_topo[i] = z[lnk]
    
del lat, lon, z

ncfile = nc.Dataset('../data/topo_compact.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

nfiles = ncfile.createDimension('nfiles', sz_tmp)
lat = ncfile.createDimension('lat', nlat)
lon = ncfile.createDimension('lon', nlon)
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact GMTED2010 USGS Topography grid for testing and debugging purposes'
print(ncfile.title)

lat = ncfile.createVariable('lat', np.float32, ('nfiles','lat'))
lat.units = 'degrees'

lon = ncfile.createVariable('lon', np.float32, ('nfiles','lon'))
lon.units = 'degress'

topo = ncfile.createVariable('topo', np.float32, ('nfiles','lat','lon'))
topo.units = 'meters'

lat[:,:] = compactified_lat
lon[:,:] = compactified_lon
topo[:,:,:] = compactified_topo

ncfile.close(); print('Dataset is closed!')

[43 46 63 67 70 91 94]
(7, 2400, 3600)
i, lnk =  (0, 43)
i, lnk =  (1, 46)
i, lnk =  (2, 63)
i, lnk =  (3, 67)
i, lnk =  (4, 70)
i, lnk =  (5, 91)
i, lnk =  (6, 94)
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('nfiles', <class 'netCDF4._netCDF4.Dimension'>: name = 'nfiles', size = 7)
('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 2400)
('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 3600)
Compact GMTED2010 USGS Topography grid for testing and debugging purposes
Dataset is closed!
